In [6]:
import os
import pandas as pd
import nibabel as nib
from tqdm import tqdm

In [7]:
lst_seg = os.listdir(r"C:\KAGGLE\MEDICINE\data\segmentations")
lst_with_size = []
lst_img = []
for img in tqdm(lst_seg):
    nii_image = nib.load(f'C:\KAGGLE\MEDICINE\data\segmentations\{img}')
    header = nii_image.header
    image_data = nii_image.get_fdata()
    
    lst_with_size.append(image_data.shape)
    lst_img.append(img)

100%|██████████| 206/206 [16:34<00:00,  4.83s/it]


In [9]:
lst_d1 = []
lst_d2 = []
lst_count_layer = []
for d1 in tqdm(os.listdir(r"C:\KAGGLE\MEDICINE\data\train_images")):
    for d2 in os.listdir(os.path.join(r"C:\KAGGLE\MEDICINE\data\train_images", d1)):
        count_layer = len(os.listdir(os.path.join(r"C:\KAGGLE\MEDICINE\data\train_images", d1, d2)))
        lst_d1.append(d1)
        lst_d2.append(d2)
        lst_count_layer.append(count_layer)

100%|██████████| 128/128 [00:01<00:00, 76.48it/s]


In [10]:
data_1 = pd.DataFrame({
    "patient_id": lst_d1,
    "series_id": lst_d2,
    "count_layer": lst_count_layer
    
})

data_2 = pd.DataFrame({
    "series_id": lst_img,
    "size": lst_with_size,

    
})

In [11]:
data_2["series_id"] = data_2["series_id"].apply(lambda x: x[:-4])
data_2["count_layers_seg"] = data_2["size"].apply(lambda x: x[2])
merged_data = data_1.merge(data_2, on='series_id', how='left')
merged_data[~merged_data.count_layers_seg.isna()]
t = merged_data[~merged_data.count_layers_seg.isna()]
t['same'] = t['count_layers_seg'] == t['count_layer']
t['same'].value_counts()

same
True    206
Name: count, dtype: int64

In [12]:
t.head()

,patient_id,series_id,count_layer,size,count_layers_seg,same
0,10004,21057,1022,"(512, 512, 1022)",1022,True
1,10004,51033,1044,"(512, 512, 1044)",1044,True
2,10217,16066,690,"(512, 512, 690)",690,True
3,10228,30522,747,"(512, 512, 747)",747,True
4,10228,40471,391,"(512, 512, 391)",391,True


In [14]:
t.to_csv("data_for_segmentation.csv", index=False)